# 정리 후 백그라운드로 실행할 예정

In [ ]:
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
from pprint import pprint

In [2]:
## data load
txts = pd.read_excel("combined_data_NLP.xlsx")
model_idxs = txts.model
txts = txts.drop(["model", "question", "attempt"], axis = 1)

## Remove punctuation
txts['answer_processed'] = txts["answer"].map(lambda x: re.sub('[,\.!?]', '', x))

## Convert the titles to lowercase
txts['answer_processed'] = txts['answer_processed'].map(lambda x: x.lower())

## Print out the first rows of txts
txts['answer_processed'].head()


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = txts.answer_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_1537297/1092903822.py:7: SyntaxWarning: invalid escape sequence '\.'
  txts['answer_processed'] = txts["answer"].map(lambda x: re.sub('[,\.!?]', '', x))


['deciding', 'whether', 'to', 'get', 'master', 'degree', 'can', 'be', 'quite', 'personal', 'journey', 'and', 've', 'seen', 'different', 'sides', 'of', 'this', 'coin', 'through', 'my', 'own', 'experience', 'and', 'that', 'of', 'friends', 'and', 'colleagues', 'personally']


In [3]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
# NLTK Stop words
nltk.download('stopwords')


stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
!python -m spacy download en_core_web_sm

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 97.9 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
['get', 'quite', 'personal', 'journey', 'see', 'different', 'side', 'coin', 'experience', 'friend', 'colleague', 'personally', 'choose', 'pursue_master', 'degree', 'right', 'complete', 'undergraduate', 'study', 'time', 'feel', 'much', 'real', 'world', 'experience', 'gain', 'want', 'dive', 'straight', 'workforce']


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1)]


In [13]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"people" + 0.012*"think" + 0.011*"thing" + 0.011*"even" + '
  '0.009*"make" + 0.009*"get" + 0.008*"way" + 0.007*"time" + 0.006*"know" + '
  '0.006*"find"'),
 (1,
  '0.014*"people" + 0.011*"think" + 0.009*"tooth" + 0.008*"even" + 0.007*"get" '
  '+ 0.006*"thing" + 0.006*"feel" + 0.006*"make" + 0.005*"day" + 0.005*"time"'),
 (2,
  '0.017*"time" + 0.008*"get" + 0.008*"make" + 0.006*"travel" + 0.006*"go" + '
  '0.006*"think" + 0.006*"even" + 0.006*"people" + 0.005*"work" + '
  '0.005*"year"'),
 (3,
  '0.014*"get" + 0.011*"make" + 0.008*"name" + 0.008*"work" + 0.007*"go" + '
  '0.007*"even" + 0.007*"day" + 0.006*"time" + 0.006*"take" + 0.005*"sound"'),
 (4,
  '0.012*"get" + 0.011*"go" + 0.010*"say" + 0.008*"time" + 0.007*"know" + '
  '0.006*"make" + 0.006*"day" + 0.006*"take" + 0.006*"see" + 0.006*"tell"'),
 (5,
  '0.013*"feel" + 0.010*"people" + 0.010*"go" + 0.009*"say" + 0.008*"think" + '
  '0.008*"know" + 0.008*"really" + 0.008*"get" + 0.008*"time" + 0.008*"make"'),
 (6,
  

In [30]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3518118387571053


`-` 이거는 다른 퍼플렉시티 같은데

In [39]:
lda_model.log_perplexity([corpus[0]])

/root/anaconda3/envs/NLP/lib/python3.12/site-packages/gensim/models/ldamodel.py:850: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


-1492.821032721594

# 그냥 코드나 리뷰하고 있자

---

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import logging
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from kneed import KneeLocator
from tqdm import tqdm

## Warning 이상의 문제 발생 시 출력
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## nltk set download
try:
    nltk.data.find('tokenizers/punkt')
except:
    nltk.download('punkt')

try:
    nltk.data.find("corpora/stopwords")
except:
    nltk.download("stopwords")

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except:
    nltk.download("averaged_perceptron_tagger")

stop_words = set(stopwords.words("english"))

def extract_english_tokens(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text) ## 구두점 전처리
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words and len(t) > 2]
    return tokens

def select_optimal_topic(coherence_scores, topic_range):
    knee = KneeLocator(topic_range, coherence_scores, curve = "concave", direction = "decreasing")
    
    if knee.knee is not None:
        print(f"KneeLocator detected optimal topic number: {knee.knee}")
        return knee.knee
    
    best_topic = topic_range[np.argmax(coherence_scores)]
    print(f"Using highest coherence: {best_topic}")
    return best_topic

def main():
    file_path = "combined_data_NLP.xlsx"
    df = pd.read_excel(file_path)
    df = df.dropna(subset = ["answer"])
    df["text"] = df["answer"]

    ## 불용어 및 토크나이즈 설정
    df["tokens"] = df["text"].apply(extract_english_tokens)
    tokens_list = df["tokens"].tolist()

    dictionary = Dictionary(tokens_list)
    corpus = [dictionary.doc2bow(tokens) for tokens in tokens_list]

    ## 최적 토픽 수 계산
    topic_range = range(2, 6)
    coherence_scores = []

    print("-----토픽 개수 산정을 위한 coherence score 계산 중-----")

    for n in topic_range:
        lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = n, passes = 10, random_state = 42, alpha = "auto")
        cm = CoherenceModel(model = lda_model, texts = tokens_list, dictionary = dictionary, coherence = "c_v")
        coherence_scores.append(cm.get_coherence())
        print(f"Topic {n} - Coherence: {coherence_scores[-1]:.4f}")

    optimal_topics = select_optimal_topic(coherence_scores, topic_range)
    print(f"최적 토픽 수: {optimal_topics}")

    ## 메인 토픽 모델 학습
    main_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = optimal_topics,
                          passes = 50,
                          random_state = 42,
                          update_every = 1,
                          eval_every = 1,
                          alpha = "auto")
    
    ## 서브토픽 분석
    subtopic_records = []
    subtopic_labels = []
    subtopic_words = []

    ## 각 최적의 토픽별로 서브토픽 추출
    for topic_id in range(optimal_topics):
        print(f"\n[Subtopic Analysis for Topic {topic_id}]")
        topic_docs = []

        ## 메인 토픽과 관련된 문서 추출
        for i, dist in enumerate(main_model[corpus]):
            for t_id, prob in dist:
                if t_id == topic_id:
                    topic_docs.append((i, prob))

        topic_docs = sorted(topic_docs, key=lambda x: x[1], reverse=True)[:20]  ## 토픽을 대표하는 상위 20개 문서 추출
        doc_indices = [idx for idx, _ in topic_docs]
        filtered_texts = [tokens_list[i] for i in doc_indices]

        ## 메인 토픽에 해당하는 문서가 하나도 없으면 다음 반복문으로 이동... 방어용 코드
        if len(filtered_texts) == 0:
            print(f"⚠️ Topic {topic_id} has no representative documents.")
            continue

        sub_dict = Dictionary(filtered_texts)
        sub_corpus = [sub_dict.doc2bow(text) for text in tokens_list]  ## 전체 문서를 sub_dict 공간으로 사영?

        # 최적 서브토픽 수 선택
        scores = []
        for n in range(2, 10):
            try:
                temp_model = LdaModel(corpus=[sub_dict.doc2bow(t) for t in filtered_texts], id2word=sub_dict, num_topics=n, passes=15, random_state=42)
                cm = CoherenceModel(model=temp_model, texts=filtered_texts, dictionary=sub_dict, coherence='c_v')
                scores.append(cm.get_coherence())
            except:
                scores.append(0)
        best_n = range(2, 10)[np.argmax(scores)]

        # 서브모델 학습
        sub_model = LdaModel(corpus=sub_corpus, id2word=sub_dict, num_topics=best_n,
                             passes=50,
                             random_state=42,
                             update_every=1,
                             eval_every=1,
                             alpha='auto')

        # 키워드 저장
        for t_id in range(best_n):
            terms = sub_model.show_topic(t_id, topn=20)
            keywords = ", ".join([f"{word} ({weight:.3f})" for word, weight in terms])
            subtopic_words.append({
                "Model": f"LDA {topic_id}",
                "Subtopic": f"{topic_id}-{t_id+1}",
                "Keywords": keywords
            })

        # 전체 문서에 대한 분포 계산
        for i, doc in enumerate(tokens_list):
            bow = sub_dict.doc2bow(doc)
            dist = sub_model.get_document_topics(bow, minimum_probability=0) #해당 문서가 서브토픽들에 대해 가지는 확률 분포
            record = {'doc_id': i}
            for t_id, prob in dist:
                label = f"{topic_id}-{t_id+1}"
                record[label] = prob
                if label not in subtopic_labels:
                    subtopic_labels.append(label)
            subtopic_records.append(record)

    # 5. 결과 저장
    result_df = pd.DataFrame(subtopic_records)
    result_df = result_df.groupby("doc_id").mean().reset_index()
    result_df["llm_name"] = df["model"]

    # 누락된 서브토픽 컬럼 0으로 채우기
    for col in subtopic_labels:
        if col not in result_df.columns:
            result_df[col] = 0.0

    result_df.to_excel("C:/Users/User/Desktop/soloplay/01. nlp/stopic_final/llm_subtopic_distribution_v2.xlsx", index=False)
    print("저장 완료: llm_subtopic_distribution.xlsx")

    words_df = pd.DataFrame(subtopic_words)
    words_df.to_excel("C:/Users/User/Desktop/soloplay/01. nlp/stopic_final/subtopic_keywords_v2.xlsx", index=False)
    print("저장 완료: subtopic_keywords.xlsx")

# 6. 실행 시작
if __name__ == '__main__':
    from multiprocessing import freeze_support
    freeze_support()
    main()

2025-09-10 11:47:52,421 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2025-09-10 11:47:53,336 : INFO : adding document #10000 to Dictionary<23014 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...>
2025-09-10 11:47:53,790 : INFO : built Dictionary<34546 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...> from 14833 documents (total 1640281 corpus positions)
2025-09-10 11:47:53,790 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<34546 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...> from 14833 documents (total 1640281 corpus positions)", 'datetime': '2025-09-10T11:47:53.790908', 'gensim': '4.3.3', 'python': '3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:48:41) [GCC 13.3.0]', 'platform': 'Linux-5.15.0-1048-nvidia-x86_64-with-glibc2.35', 'event': 'created'}
2025-09-10 11:47:54,808 : INFO : using autotuned alpha, starting with [0.5, 0.5]
2025-09-10 11:47:54,809 : I

-----토픽 개수 산정을 위한 coherence score 계산 중-----


2025-09-10 11:47:55,961 : INFO : optimized alpha [0.9009148, 0.80155915]
2025-09-10 11:47:55,963 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:47:55,971 : INFO : topic #0 (0.901): 0.013*"like" + 0.008*"one" + 0.008*"might" + 0.007*"think" + 0.007*"something" + 0.007*"people" + 0.006*"also" + 0.006*"time" + 0.006*"even" + 0.006*"really"
2025-09-10 11:47:55,972 : INFO : topic #1 (0.802): 0.011*"like" + 0.009*"one" + 0.008*"people" + 0.007*"think" + 0.006*"even" + 0.006*"time" + 0.006*"life" + 0.006*"might" + 0.005*"often" + 0.005*"way"
2025-09-10 11:47:55,972 : INFO : topic diff=3.478361, rho=1.000000
2025-09-10 11:47:55,973 : INFO : PROGRESS: pass 0, at document #4000/14833
2025-09-10 11:47:57,117 : INFO : optimized alpha [1.1482253, 0.9960377]
2025-09-10 11:47:57,120 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:47:57,122 : INFO : topic #0 (1.148): 0.014*"like" + 0.010*"think" + 0.009*"one" + 0.00

Topic 2 - Coherence: 0.3726


2025-09-10 11:49:04,805 : INFO : optimized alpha [0.61386144, 0.5548838, 0.55519634]
2025-09-10 11:49:04,809 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:49:04,813 : INFO : topic #0 (0.614): 0.013*"like" + 0.009*"one" + 0.009*"might" + 0.008*"people" + 0.007*"think" + 0.007*"something" + 0.006*"really" + 0.006*"time" + 0.006*"even" + 0.006*"also"
2025-09-10 11:49:04,814 : INFO : topic #1 (0.555): 0.011*"like" + 0.010*"one" + 0.009*"people" + 0.007*"even" + 0.006*"think" + 0.006*"life" + 0.006*"time" + 0.006*"might" + 0.005*"often" + 0.005*"way"
2025-09-10 11:49:04,815 : INFO : topic #2 (0.555): 0.012*"like" + 0.008*"think" + 0.007*"something" + 0.006*"one" + 0.006*"might" + 0.006*"time" + 0.006*"would" + 0.006*"also" + 0.005*"could" + 0.005*"even"
2025-09-10 11:49:04,815 : INFO : topic diff=3.914534, rho=1.000000
2025-09-10 11:49:04,816 : INFO : PROGRESS: pass 0, at document #4000/14833
2025-09-10 11:49:05,959 : INFO : optimized alpha [0.78

...

2025-09-10 11:50:17,987 : INFO : optimized alpha [0.460262, 0.41442758, 0.41854858, 0.4246746]
2025-09-10 11:50:17,992 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:50:17,997 : INFO : topic #0 (0.460): 0.012*"like" + 0.009*"one" + 0.009*"might" + 0.008*"people" + 0.007*"something" + 0.007*"think" + 0.007*"also" + 0.006*"even" + 0.006*"time" + 0.006*"really"
2025-09-10 11:50:17,998 : INFO : topic #1 (0.414): 0.010*"like" + 0.010*"one" + 0.009*"people" + 0.007*"even" + 0.006*"life" + 0.006*"think" + 0.006*"time" + 0.006*"might" + 0.005*"often" + 0.005*"way"
2025-09-10 11:50:17,999 : INFO : topic #2 (0.419): 0.011*"like" + 0.008*"think" + 0.008*"something" + 0.006*"one" + 0.006*"also" + 0.006*"time" + 0.006*"would" + 0.006*"might" + 0.006*"could" + 0.005*"even"
2025-09-10 11:50:17,999 : INFO : topic #3 (0.425): 0.014*"like" + 0.009*"one" + 0.008*"think" + 0.007*"might" + 0.007*"people" + 0.006*"really" + 0.006*"time" + 0.006*"even" + 0.005*"way

Topic 4 - Coherence: 0.3320


2025-09-10 11:51:27,528 : INFO : optimized alpha [0.36779004, 0.32840165, 0.3322057, 0.3351854, 0.32000184]
2025-09-10 11:51:27,534 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:51:27,540 : INFO : topic #0 (0.368): 0.012*"like" + 0.009*"one" + 0.009*"might" + 0.008*"something" + 0.007*"people" + 0.007*"think" + 0.007*"even" + 0.007*"time" + 0.006*"also" + 0.006*"really"
2025-09-10 11:51:27,541 : INFO : topic #1 (0.328): 0.010*"one" + 0.010*"like" + 0.009*"people" + 0.007*"even" + 0.007*"life" + 0.006*"think" + 0.006*"time" + 0.006*"might" + 0.005*"often" + 0.005*"really"
2025-09-10 11:51:27,542 : INFO : topic #2 (0.332): 0.011*"like" + 0.009*"think" + 0.008*"something" + 0.006*"would" + 0.006*"time" + 0.006*"might" + 0.006*"one" + 0.006*"also" + 0.005*"could" + 0.005*"even"
2025-09-10 11:51:27,542 : INFO : topic #3 (0.335): 0.013*"like" + 0.009*"one" + 0.008*"think" + 0.008*"might" + 0.007*"people" + 0.007*"really" + 0.007*"time" + 0.006*"ev

Topic 5 - Coherence: 0.3217
KneeLocator detected optimal topic number: 5
최적 토픽 수: 5


2025-09-10 11:52:39,515 : INFO : -11.043 per-word bound, 2110.5 perplexity estimate based on a held-out corpus of 2000 documents with 288390 words
2025-09-10 11:52:39,517 : INFO : PROGRESS: pass 0, at document #2000/14833
2025-09-10 11:52:40,732 : INFO : optimized alpha [0.36752695, 0.32752627, 0.33193612, 0.33693862, 0.320106]
2025-09-10 11:52:40,738 : INFO : merging changes from 2000 documents into a model of 14833 documents
2025-09-10 11:52:40,744 : INFO : topic #0 (0.368): 0.012*"like" + 0.009*"one" + 0.009*"might" + 0.008*"something" + 0.007*"people" + 0.007*"think" + 0.007*"even" + 0.007*"also" + 0.006*"time" + 0.006*"really"
2025-09-10 11:52:40,745 : INFO : topic #1 (0.328): 0.010*"one" + 0.010*"like" + 0.009*"people" + 0.007*"even" + 0.006*"life" + 0.006*"think" + 0.006*"time" + 0.006*"might" + 0.005*"often" + 0.005*"really"
2025-09-10 11:52:40,746 : INFO : topic #2 (0.332): 0.011*"like" + 0.009*"think" + 0.008*"something" + 0.006*"would" + 0.006*"time" + 0.006*"one" + 0.006*"m

In [2]:
file_path = "combined_data_NLP.xlsx"
df = pd.read_excel(file_path)
df = df.dropna(subset = ["answer"])
df["text"] = df["answer"]

## 불용어 및 토크나이즈 설정
df["tokens"] = df["text"].apply(extract_english_tokens)
tokens_list = df["tokens"].tolist()

dictionary = Dictionary(tokens_list)
corpus = [dictionary.doc2bow(tokens) for tokens in tokens_list]

2025-09-10 12:04:42,818 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2025-09-10 12:04:43,710 : INFO : adding document #10000 to Dictionary<23014 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...>
2025-09-10 12:04:44,131 : INFO : built Dictionary<34546 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...> from 14833 documents (total 1640281 corpus positions)
2025-09-10 12:04:44,132 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<34546 unique tokens: ['academia', 'academic', 'achieve', 'actually', 'advanced']...> from 14833 documents (total 1640281 corpus positions)", 'datetime': '2025-09-10T12:04:44.131996', 'gensim': '4.3.3', 'python': '3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:48:41) [GCC 13.3.0]', 'platform': 'Linux-5.15.0-1048-nvidia-x86_64-with-glibc2.35', 'event': 'created'}


In [3]:
df_temp = pd.DataFrame([dist for dist in main_model[corpus]])
df_temp.to_csv("temp_dist.csv", index = False)
dup = df_temp.copy()

In [9]:
import pandas as pd
from copy import deepcopy

df_temp = pd.read_csv("temp_dist.csv")
dup = deepcopy(df_temp)

TypeError: read_csv() got an unexpected keyword argument 'index'

In [7]:
for i in range(len(df_temp)) :
    for j in range(5) :
        if dup.iloc[i, j] != None :
            df_temp[i, j] = 0 ## 현재 자리 0으로
            try :
                df_temp.iloc[i, dup.iloc[i, j][0]] = dup.iloc[i, j][1] ## 원래 자리 확률값으로
            except :
                print(i, j)
                print(dup.iloc[i, j])
                break

0 0
0
1 0
1
2 0
2
3 0
3
4 0
4
5 0
5
6 0
6
7 0
7
8 0
8
9 0
9
10 0
10
11 0
11
12 0
12
13 0
13
14 0
14
15 0
15
16 0
16
17 0
17
18 0
18
19 0
19
20 0
20
21 0
21
22 0
22
23 0
23
24 0
24
25 0
25
26 0
26
27 0
27
28 0
28
29 0
29
30 0
30
31 0
31
32 0
32
33 0
33
34 0
34
35 0
35
36 0
36
37 0
37
38 0
38
39 0
39
40 0
40
41 0
41
42 0
42
43 0
43
44 0
44
45 0
45
46 0
46
47 0
47
48 0
48
49 0
49
50 0
50
51 0
51
52 0
52
53 0
53
54 0
54
55 0
55
56 0
56
57 0
57
58 0
58
59 0
59
60 0
60
61 0
61
62 0
62
63 0
63
64 0
64
65 0
65
66 0
66
67 0
67
68 0
68
69 0
69
70 0
70
71 0
71
72 0
72
73 0
73
74 0
74
75 0
75
76 0
76
77 0
77
78 0
78
79 0
79
80 0
80
81 0
81
82 0
82
83 0
83
84 0
84
85 0
85
86 0
86
87 0
87
88 0
88
89 0
89
90 0
90
91 0
91
92 0
92
93 0
93
94 0
94
95 0
95
96 0
96
97 0
97
98 0
98
99 0
99
100 0
100
101 0
101
102 0
102
103 0
103
104 0
104
105 0
105
106 0
106
107 0
107
108 0
108
109 0
109
110 0
110
111 0
111
112 0
112
113 0
113
114 0
114
115 0
115
116 0
116
117 0
117
118 0
118
119 0
119
120 0
120
121 0
121


/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe

518 0
518
519 0
519
520 0
520
521 0
521
522 0
522
523 0
523
524 0
524
525 0
525
526 0
526
527 0
527
528 0
528
529 0
529
530 0
530
531 0
531
532 0
532
533 0
533
534 0
534
535 0
535
536 0
536
537 0
537
538 0
538
539 0
539
540 0
540
541 0
541
542 0
542
543 0
543
544 0
544
545 0
545
546 0
546
547 0
547
548 0
548
549 0
549
550 0
550
551 0
551
552 0
552
553 0
553
554 0
554
555 0
555
556 0
556
557 0
557
558 0
558
559 0
559
560 0
560
561 0
561
562 0
562
563 0
563
564 0
564
565 0
565
566 0
566
567 0
567
568 0
568
569 0
569
570 0
570
571 0
571
572 0
572
573 0
573
574 0
574
575 0
575
576 0
576
577 0
577
578 0
578
579 0
579
580 0
580
581 0
581
582 0
582
583 0
583
584 0
584
585 0
585
586 0
586
587 0
587
588 0
588
589 0
589
590 0
590
591 0
591
592 0
592
593 0
593
594 0
594
595 0
595
596 0
596
597 0
597
598 0
598
599 0
599
600 0
600
601 0
601
602 0
602
603 0
603
604 0
604
605 0
605
606 0
606
607 0
607
608 0
608
609 0
609
610 0
610
611 0
611
612 0
612
613 0
613
614 0
614
615 0
615
616 0
616
617 0
617


/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe

941 0
941
942 0
942
943 0
943
944 0
944
945 0
945
946 0
946
947 0
947
948 0
948
949 0
949
950 0
950
951 0
951
952 0
952
953 0
953
954 0
954
955 0
955
956 0
956
957 0
957
958 0
958
959 0
959
960 0
960
961 0
961
962 0
962
963 0
963
964 0
964
965 0
965
966 0
966
967 0
967
968 0
968
969 0
969
970 0
970
971 0
971
972 0
972
973 0
973
974 0
974
975 0
975
976 0
976
977 0
977
978 0
978
979 0
979
980 0
980
981 0
981
982 0
982
983 0
983
984 0
984
985 0
985
986 0
986
987 0
987
988 0
988
989 0
989
990 0
990
991 0
991
992 0
992
993 0
993
994 0
994
995 0
995
996 0
996
997 0
997
998 0
998
999 0
999
1000 0
1000
1001 0
1001
1002 0
1002
1003 0
1003
1004 0
1004
1005 0
1005
1006 0
1006
1007 0
1007
1008 0
1008
1009 0
1009
1010 0
1010
1011 0
1011
1012 0
1012
1013 0
1013
1014 0
1014
1015 0
1015
1016 0
1016
1017 0
1017
1018 0
1018
1019 0
1019
1020 0
1020
1021 0
1021
1022 0
1022
1023 0
1023
1024 0
1024
1025 0
1025
1026 0
1026
1027 0
1027
1028 0
1028
1029 0
1029
1030 0
1030
1031 0
1031
1032 0
1032
1033 0
1033
10

...

/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe

4939 0
4939
4940 0
4940
4941 0
4941
4942 0
4942
4943 0
4943
4944 0
4944
4945 0
4945
4946 0
4946
4947 0
4947
4948 0
4948
4949 0
4949
4950 0
4950
4951 0
4951
4952 0
4952
4953 0
4953
4954 0
4954
4955 0
4955
4956 0
4956
4957 0
4957
4958 0
4958
4959 0
4959
4960 0
4960
4961 0
4961
4962 0
4962
4963 0
4963
4964 0
4964
4965 0
4965
4966 0
4966
4967 0
4967
4968 0
4968
4969 0
4969
4970 0
4970
4971 0
4971
4972 0
4972
4973 0
4973
4974 0
4974
4975 0
4975
4976 0
4976
4977 0
4977
4978 0
4978
4979 0
4979
4980 0
4980
4981 0
4981
4982 0
4982
4983 0
4983
4984 0
4984
4985 0
4985
4986 0
4986
4987 0
4987
4988 0
4988
4989 0
4989
4990 0
4990
4991 0
4991
4992 0
4992
4993 0
4993
4994 0
4994
4995 0
4995
4996 0
4996
4997 0
4997
4998 0
4998
4999 0
4999
5000 0
5000
5001 0
5001
5002 0
5002
5003 0
5003
5004 0
5004
5005 0
5005
5006 0
5006
5007 0
5007
5008 0
5008
5009 0
5009
5010 0
5010
5011 0
5011
5012 0
5012
5013 0
5013
5014 0
5014
5015 0
5015
5016 0
5016
5017 0
5017
5018 0
5018
5019 0
5019
5020 0
5020
5021 0
5021
5022

/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp[i, j] = 0 ## 현재 자리 0으로
/tmp/ipykernel_1553722/2428675678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe

5095 0
5095


KeyboardInterrupt: 

In [17]:
[dist for dist in main_model[corpus]]

[[(0, 0.8691078), (1, 0.12751621)],
 [(0, 0.98204076), (3, 0.016536381)],
 [(0, 0.89222616), (1, 0.060337104), (2, 0.014186196), (3, 0.032855492)],
 [(0, 0.9511336), (1, 0.044832963)],
 [(0, 0.9299507), (1, 0.06666628)],
 [(0, 0.9735764), (3, 0.02522781)],
 [(0, 0.9095308), (1, 0.058316592), (2, 0.022097778)],
 [(0, 0.9501818), (1, 0.045875773)],
 [(0, 0.89087325), (1, 0.054906517), (3, 0.05346034)],
 [(0, 0.95735043), (1, 0.03843825)],
 [(0, 0.955734), (3, 0.033920832)],
 [(0, 0.97862506), (3, 0.020014388)],
 [(0, 0.8954746), (1, 0.067486495), (3, 0.036160335)],
 [(0, 0.970366), (3, 0.028437342)],
 [(0, 0.9957849)],
 [(0, 0.9512731), (1, 0.04475538)],
 [(0, 0.9540714), (1, 0.042237263)],
 [(0, 0.9671737), (3, 0.03141432)],
 [(0, 0.94888276), (1, 0.036503393), (2, 0.011969951)],
 [(0, 0.9955138)],
 [(0, 0.9560205), (1, 0.033269655)],
 [(0, 0.89161396), (1, 0.07836879), (3, 0.02921693)],
 [(0, 0.99585915)],
 [(0, 0.96598995), (1, 0.030503968)],
 [(0, 0.97807455), (3, 0.020660466)],
 [(0

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import logging
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from kneed import KneeLocator
from tqdm import tqdm

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


def extract_english_tokens(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words and len(t) > 2]
    return tokens

def select_optimal_topic(coherence_scores, topic_range):
    knee = KneeLocator(topic_range, coherence_scores, curve="concave", direction="decreasing")
    if knee.knee is not None:
        print(f"KneeLocator detected optimal topic number: {knee.knee}")
        return knee.knee
    best_topic = topic_range[np.argmax(coherence_scores)]
    print(f"Using highest coherence: {best_topic}")
    return best_topic

def main():
    file_path = "C:/Users/User/Desktop/soloplay/01. nlp/combined_data_NLP.xlsx"
    df = pd.read_excel(file_path)
    df = df.dropna(subset=['answer'])
    df['text'] = df['answer']

    # 불용어 및 토큰화 설정
    nltk.download('punkt')
    nltk.download('stopwords')
    global stop_words
    stop_words = set(stopwords.words('english'))

    df['tokens'] = df['text'].apply(extract_english_tokens)
    tokens_list = df['tokens'].tolist()

    dictionary = Dictionary(tokens_list)
    corpus = [dictionary.doc2bow(tokens) for tokens in tokens_list]

    # 2. 최적 토픽 수 계산
    topic_range = range(2, 6)
    coherence_scores = []

    print("토픽 갯수 산정을 위한 coherence score 계산중")
    for n in topic_range:
        lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=n, passes=10, random_state=42, alpha='auto')
        cm = CoherenceModel(model=lda_model, texts=tokens_list, dictionary=dictionary, coherence='c_v')
        coherence_scores.append(cm.get_coherence())
        print(f"Topic {n} - Coherence: {coherence_scores[-1]:.4f}")

    optimal_topics = select_optimal_topic(coherence_scores, topic_range)
    print(f"최적 토픽 수: {optimal_topics}")

    # 3. 메인 토픽 모델 학습
    main_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=optimal_topics,
                          passes=50,
                          random_state=42,
                          update_every=1,
                          eval_every=1,
                          alpha='auto')

    # 4. 서브토픽 분석
    subtopic_records = []
    subtopic_labels = []
    subtopic_words = []

    for topic_id in range(optimal_topics):
        print(f"\n[Subtopic Analysis for Topic {topic_id}]")
        topic_docs = []
        for i, dist in enumerate(main_model[corpus]):
            for t_id, prob in dist:
                if t_id == topic_id:
                    topic_docs.append((i, prob))
        topic_docs = sorted(topic_docs, key=lambda x: x[1], reverse=True)[:20]
        doc_indices = [idx for idx, _ in topic_docs]
        filtered_texts = [tokens_list[i] for i in doc_indices]

        if len(filtered_texts) == 0:
            print(f"⚠️ Topic {topic_id} has no representative documents.")
            continue

        sub_dict = Dictionary(filtered_texts)
        sub_corpus = [sub_dict.doc2bow(text) for text in tokens_list]  # 전체 문서 대상

        # 최적 서브토픽 수 선택
        scores = []
        for n in range(2, 10):
            try:
                temp_model = LdaModel(corpus=[sub_dict.doc2bow(t) for t in filtered_texts], id2word=sub_dict, num_topics=n, passes=15, random_state=42)
                cm = CoherenceModel(model=temp_model, texts=filtered_texts, dictionary=sub_dict, coherence='c_v')
                scores.append(cm.get_coherence())
            except:
                scores.append(0)
        best_n = range(2, 10)[np.argmax(scores)]

        # 서브모델 학습
        sub_model = LdaModel(corpus=sub_corpus, id2word=sub_dict, num_topics=best_n,
                             passes=50,
                             random_state=42,
                             update_every=1,
                             eval_every=1,
                             alpha='auto')

        # 키워드 저장
        for t_id in range(best_n):
            terms = sub_model.show_topic(t_id, topn=20)
            keywords = ", ".join([f"{word} ({weight:.3f})" for word, weight in terms])
            subtopic_words.append({
                "Model": f"LDA {topic_id}",
                "Subtopic": f"{topic_id}-{t_id+1}",
                "Keywords": keywords
            })

        # 전체 문서에 대한 분포 계산
        for i, doc in enumerate(tokens_list):
            bow = sub_dict.doc2bow(doc)
            dist = sub_model.get_document_topics(bow, minimum_probability=0) #해당 문서가 서브토픽들에 대해 가지는 확률 분포
            record = {'doc_id': i}
            for t_id, prob in dist:
                label = f"{topic_id}-{t_id+1}"
                record[label] = prob
                if label not in subtopic_labels:
                    subtopic_labels.append(label)
            subtopic_records.append(record)

    # 5. 결과 저장
    result_df = pd.DataFrame(subtopic_records)
    result_df = result_df.groupby("doc_id").mean().reset_index()
    result_df["llm_name"] = df["model"]

    # 누락된 서브토픽 컬럼 0으로 채우기
    for col in subtopic_labels:
        if col not in result_df.columns:
            result_df[col] = 0.0

    result_df.to_excel("C:/Users/User/Desktop/soloplay/01. nlp/stopic_final/llm_subtopic_distribution_v2.xlsx", index=False)
    print("저장 완료: llm_subtopic_distribution.xlsx")

    words_df = pd.DataFrame(subtopic_words)
    words_df.to_excel("C:/Users/User/Desktop/soloplay/01. nlp/stopic_final/subtopic_keywords_v2.xlsx", index=False)
    print("저장 완료: subtopic_keywords.xlsx")

# 6. 실행 시작
if __name__ == '__main__':
    from multiprocessing import freeze_support
    freeze_support()
    main()